In [1]:
import polars as pl
import joblib 

In [2]:
inference_df = pl.scan_parquet("train_data/inference_final_mix.parquet")

In [3]:
# 2. Load ground truth
gt = joblib.load("train_data/final_groundtruth_dict.pkl")

In [4]:
# 3. Kiểm tra cấu trúc của ground truth
print("Ground truth type:", type(gt))
print("Ground truth keys:" if isinstance(gt, dict) else "Ground truth shape:")
if isinstance(gt, dict):
    print(list(gt.keys())[:5])
    # Lấy ra danh sách customer IDs từ groundtruth
    if isinstance(list(gt.keys())[0], (int, str)):
        gt_customers = set(gt.keys())
        print(f"Số lượng khách hàng trong groundtruth: {len(gt_customers)}")
else:
    print(gt.shape if hasattr(gt, 'shape') else len(gt))

Ground truth type: <class 'dict'>
Ground truth keys:
[2337685, 7934799, 2052333, 6548920, 368770]
Số lượng khách hàng trong groundtruth: 644970


In [5]:
# 4. Kiểm tra cấu trúc của inference_df
print("Inference columns:", inference_df.collect_schema().names()[:10])
print("\nInference shape (estimated):", inference_df.select(pl.len()).collect())

Inference columns: ['customer_id', 'item_id', 'feat1_customer_item_freq', 'feat2_brand_affinity', 'feat2_type_affinity', 'feat3_urgency_score', 'feat3_is_in_window', 'feat4_pop_30d_log', 'feat4_pop_trend', 'feat4_pop_category_rank']

Inference shape (estimated): shape: (1, 1)
┌───────────┐
│ len       │
│ ---       │
│ u32       │
╞═══════════╡
│ 256561832 │
└───────────┘


In [6]:
# 5. Lấy danh sách customer_id unique từ inference_df
inference_customers = inference_df.select("customer_id").unique().collect()["customer_id"].to_list()
print(f"Số lượng khách hàng unique trong inference: {len(inference_customers)}")

Số lượng khách hàng unique trong inference: 2569977


In [7]:
# 6. Tìm các khách hàng có trong cả inference và groundtruth
inference_customers_set = set(inference_customers)
common_customers = list(gt_customers.intersection(inference_customers_set))
print(f"Số lượng khách hàng chung: {len(common_customers)}")

Số lượng khách hàng chung: 509735


In [8]:
# 7. Chọn ngẫu nhiên 15,000 khách hàng từ danh sách chung
import random
random.seed(42)  # Để reproducible
sampled_customers = random.sample(common_customers, 15000)
print(f"Đã chọn {len(sampled_customers)} khách hàng")

Đã chọn 15000 khách hàng


In [9]:
# 8. Lọc inference_df để chỉ giữ lại 15,000 khách hàng đã chọn
inference_df_small = inference_df.filter(pl.col("customer_id").is_in(sampled_customers))

# Collect để xem kích thước
result_size = inference_df_small.select(pl.len()).collect()
print(f"Số lượng rows trong inference_df_small: {result_size['len'][0]:,}")

# Kiểm tra số lượng khách hàng unique
unique_customers = inference_df_small.select("customer_id").unique().collect().height
print(f"Số lượng khách hàng unique: {unique_customers}")

Số lượng rows trong inference_df_small: 1,450,359
Số lượng khách hàng unique: 15000


In [10]:
# 9. Tạo groundtruth nhỏ cho 15,000 khách hàng đã chọn
gt_small = {customer_id: gt[customer_id] for customer_id in sampled_customers}
print(f"Số lượng khách hàng trong groundtruth nhỏ: {len(gt_small)}")
print(f"Ví dụ một entry trong groundtruth: {list(gt_small.items())[0]}")

Số lượng khách hàng trong groundtruth nhỏ: 15000
Ví dụ một entry trong groundtruth: (4901623, ['6697000000003', '4690000000001'])


In [11]:
# 10. Lưu inference_df nhỏ vào file parquet
output_path_inference = "train_data/inference_final_small.parquet"
inference_df_small.collect().write_parquet(output_path_inference)
print(f"Đã lưu inference nhỏ vào: {output_path_inference}")

Đã lưu inference nhỏ vào: train_data/inference_final_small.parquet


In [12]:
# 11. Lưu groundtruth nhỏ vào file pickle
output_path_gt = "train_data/groundtruth_small.pkl"
joblib.dump(gt_small, output_path_gt)
print(f"Đã lưu groundtruth nhỏ vào: {output_path_gt}")

Đã lưu groundtruth nhỏ vào: train_data/groundtruth_small.pkl


In [13]:
# 12. Summary
print("=" * 60)
print("TÓM TẮT KẾT QUẢ")
print("=" * 60)
print(f"\n📊 DỮ LIỆU GỐC:")
print(f"   - Inference: 244,356,758 rows (2,442,305 khách hàng)")
print(f"   - Groundtruth: 391,900 khách hàng")
print(f"   - Khách hàng chung: 333,720 khách hàng")
print(f"\n📦 DỮ LIỆU MỚI (ĐÃ LƯU):")
print(f"   - Inference nhỏ: {result_size['len'][0]:,} rows (15,000 khách hàng)")
print(f"   - Groundtruth nhỏ: 15,000 khách hàng")
print(f"   - Tỷ lệ giảm: {(1 - result_size['len'][0] / 244356758) * 100:.1f}% (inference)")
print(f"\n💾 VỊ TRÍ FILES:")
print(f"   - {output_path_inference}")
print(f"   - {output_path_gt}")
print(f"\n✅ Bạn có thể dùng 2 files này để tune hyperparameters nhanh hơn!")
print("=" * 60)

TÓM TẮT KẾT QUẢ

📊 DỮ LIỆU GỐC:
   - Inference: 244,356,758 rows (2,442,305 khách hàng)
   - Groundtruth: 391,900 khách hàng
   - Khách hàng chung: 333,720 khách hàng

📦 DỮ LIỆU MỚI (ĐÃ LƯU):
   - Inference nhỏ: 1,450,359 rows (15,000 khách hàng)
   - Groundtruth nhỏ: 15,000 khách hàng
   - Tỷ lệ giảm: 99.4% (inference)

💾 VỊ TRÍ FILES:
   - train_data/inference_final_small.parquet
   - train_data/groundtruth_small.pkl

✅ Bạn có thể dùng 2 files này để tune hyperparameters nhanh hơn!
